# Трюки для обучения нейронных сетей

### **1. Аугментации**

На практике часто оказывается, что нет достаточно большого количества размеченных данных, чтобы адекватно обучить модель, а общедоступные датасеты плохо приближают генеральную совокупность данных, возникающую в реальной задаче.

Однако в таких ситуациях может оказаться полезным изменить объекты из доступной нам выборки. Например, если речь идёт о работе с изображениями, картинки — в зависимости от конкретной задачи — можно поворачивать, искривлять, накладывать дополнительные объекты, добавлять шум, менять яркость, расфокусировать и т. д.

Многие из таких функций реализованы в [preprocessing.image](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image) из TensorFlow.

Возьмём произвольный объект из датасета CIFAR-10:

In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

In [0]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

rand_index = np.random.randint(0, len(y_train))

x_cur = x_train[rand_index,:]
y_cur = y_train[rand_index, 0]

def draw_image(x, y):
  plt.imshow(x)
  plt.title(cifar10_classes[y])
  plt.show()
  
draw_image(x_cur, y_cur)

In [0]:
x = tf.keras.preprocessing.image.apply_affine_transform(x_cur, theta=30)
draw_image(x, y_cur)
#Повернём картинку на 30 градусов

In [0]:
x = tf.keras.preprocessing.image.apply_affine_transform(x_cur, shear=35)
draw_image(x, y_cur)
#Наклоним картинку на 35 градусов

In [0]:
x = tf.keras.preprocessing.image.apply_brightness_shift(x_cur, 0.5)
draw_image(x.astype(int), y_cur)
#уменьшим яркость

Попробуйте реализовать зеркальное отображение картинки относительно вертикальной оси (подсказка: в numpy есть функция для изменения порядка элементов в массиве на обратный по заданному измерению):

In [0]:
def horizontal_reflect(image):
  #YOUR CODE HERE

draw_image(horizontal_reflect(x_cur), y_cur)

Обучим на CIFAR-10 свёрточную нейронную сеть без аугментации (см. семинар 4).

In [0]:
x_train2 = x_train / 255 - 0.5
x_test2 = x_test / 255 - 0.5

y_train2 = tf.keras.utils.to_categorical(y_train, 10)
y_test2 = tf.keras.utils.to_categorical(y_test, 10)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, LeakyReLU

def make_model():
    model = Sequential()
    model.add(Conv2D(filters=16, padding='same', kernel_size=(3,3), input_shape=(32,32,3)))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(filters=32, padding='same', kernel_size=(3,3)))  
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Conv2D(filters=64, padding='same', kernel_size=(3,3)))  
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same')) 
    model.add(Flatten())
    model.add(Dense(256))                
    model.add(LeakyReLU(0.1))
    model.add(Dense(10))             
    model.add(Activation("softmax"))
    
    return model

In [0]:
def reset_tf_session():
    curr_session = tf.get_default_session()
    if curr_session is not None:
        curr_session.close()
    tf.keras.backend.clear_session()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    tf.keras.backend.set_session(s)
    return s

In [0]:
def train_model(x_train, y_train, x_test, y_test, make_model):
  s = reset_tf_session()
  model = make_model()
  INIT_LR = 5e-3
  BATCH_SIZE = 32
  EPOCHS = 10
  model.compile(
      loss='categorical_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=INIT_LR),
      metrics=['accuracy']
  )

  def lr_scheduler(epoch):
      return INIT_LR * 0.9 ** epoch
  
  model.fit(
      x_train, y_train,
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler)],
      validation_data=(x_test, y_test),
      shuffle=True,
      verbose=1,
      initial_epoch=0
  )
  
train_model(x_train2, y_train2, x_test2, y_test2, make_model)



Добавим аугментацию, используя ImageDataGenerator из preprocessing.image.

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(#YOUR CODE HERE)

datagen.fit(x_train2)

BATCH_SIZE = 32

def train_gen_model(train_gen, x_test, y_test, make_model):
  s = reset_tf_session()
  model = make_model()
  INIT_LR = 5e-3
  EPOCHS = 10
  model.compile(
      loss='categorical_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=INIT_LR),
      metrics=['accuracy']
  )

  def lr_scheduler(epoch):
      return INIT_LR * 0.9 ** epoch
  
  model.fit_generator(
      train_gen,
      steps_per_epoch = len(x_train) / BATCH_SIZE,
      epochs=EPOCHS,
      callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler)],
      validation_data=(x_test, y_test),
      shuffle=True,
      verbose=1,
      initial_epoch=0
  )
  
train_gen_model(datagen.flow(x_train2, y_train2, batch_size=BATCH_SIZE), x_test2, y_test2, make_model)



Обратите внимание, что происходит с loss и accuracy на валидации. Сравните с предыдущим результатом.

### **2. Dropout**



При обучении моделей из предыдущего пункта можно заметить, что после нескольких эпох функция потерь на тестовой выборке начинает расти (при использовании аугментации это происходит позже; как вы думаете, почему?), то есть происходит переобучение. Чтобы избежать этой проблемы, существуют различные методы, например, [Dropout](https://http://jmlr.org/papers/v15/srivastava14a.html).

Идея метода заключается в том, чтобы при обучении случайно исключать из некоторых слоёв часть нейронов. Предполагается, что это уменьшит "совместную адаптацию" (когда одни нейроны обучаются исправлять ошибки других) и позволит избежать переобучения.

Добавим в нашу модель слои Dropout.

In [0]:
def make_dropout_model():
    model = Sequential()
    #YOUR CODE HERE
    #используйте модель из make_model и добавьте между её слоями слои Dropout(p), где p - вероятность исключения одного нейрона
   
    return model
  
train_model(x_train2, y_train2, x_test2, y_test2, make_dropout_model)

Попробуйте поменять параметр Dropout. Как это влияет на результат?

### **3. Batch-нормализация**

Как вы наверняка помните, нейронным сетям легче обучаться на нормализованных данных, однако после прохождения через несколько слоёв сети матожидание и дисперсия сигнала, попадающего на вход слежующим слоям, могут значительно искажаться и таким образом вносить несоответствие в градиенты разных слоёв.

Метод [Batch normalization](https://arxiv.org/abs/1502.03167) предлагает нормализовать данные не только на входе, но и внутри сети.

Добавим в модель слои BatchNormalization.

In [0]:
from tensorflow.keras.layers import BatchNormalization

def make_batchnorm_model():
    #YOUR CODE HERE
    #используйте модель из make_model и добавьте между её слоями слои BatchNormalization()
    
    return model
  
train_model(x_train2, y_train2, x_test2, y_test2, make_batchnorm_model)

### **4. L2 - регуляризация**

Так же, как в случае других моделей машинного обучения, нейронные сети можно штрафовать за слишком большие веса, добавляя к функции потерь норму весов с некоторым коэффициентом. Добавить регуляризацию по весам слоя можно установкой параметра kernel_regularizer.

In [0]:
def make_model_with_l2():
    model = Sequential()
    #YOUR CODE HERE
    #используйте модель из make_model и добавьте в параметры слоёв l2-регуляризацию с помощью параметра kernel_regularizer
    #подсказка: поищите подходящий регуляризатор в tf.keras.regularizers
    
    return model

train_model(x_train2, y_train2, x_test2, y_test2, make_model_with_l2)

Что будет, если увеличить коэффициент перед весами? А если уменьшить?

P. S. Обратите внимание, что регуляризация замедляет скорость обучения. Улучшится ли результат, если увеличить количество эпох?